In [78]:
import torch
from botorch.models import SingleTaskGP, FixedNoiseGP
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound, qExpectedImprovement
from botorch.optim import optimize_acqf


train_X = torch.rand(10, 5)
Y = 1 - (train_X - 0.5).norm(dim=-1, keepdim=True)  # explicit output dimension
Y += 0.1 * torch.rand_like(Y)
train_Y = (Y - Y.mean()) / Y.std()

print(train_X.shape, train_Y.shape)

train_Y_var = torch.full_like(train_Y, 0.2)
gp = FixedNoiseGP(train_X, train_Y, train_Y_var)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll);

bounds = torch.stack([torch.zeros(5), torch.ones(5)])

torch.Size([10, 5]) torch.Size([10, 1])


In [80]:
UCB = UpperConfidenceBound(gp, beta=0.1)
candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=1, num_restarts=15, raw_samples=256
)
print(candidate.shape)
print(acq_value)

torch.Size([1, 5])
tensor(1.3541)


In [81]:
qEI = qExpectedImprovement(gp, best_f=0.2)
candidate, acq_value = optimize_acqf(
    qEI, bounds=bounds, q=3, num_restarts=15, raw_samples=256, sequential=True
)
print(candidate.shape)
print(acq_value)

candidate, acq_value = optimize_acqf(
    qEI, bounds=bounds, q=3, num_restarts=15, raw_samples=256, sequential=False
)
print(candidate.shape)
print(acq_value)

torch.Size([3, 5])
tensor([1.0025, 1.1652, 1.2731])
torch.Size([3, 5])
tensor(1.3035)


In [48]:
qEI = qExpectedImprovement(gp, best_f=train_Y.max())
qei = qEI(train_X)
print(qei)

tensor([0.0221], grad_fn=<MeanBackward1>)


In [82]:
import gpytorch.kernels

In [83]:
help(gpytorch.kernels)

Help on package gpytorch.kernels in gpytorch:

NAME
    gpytorch.kernels

PACKAGE CONTENTS
    additive_structure_kernel
    arc_kernel
    cosine_kernel
    cylindrical_kernel
    distributional_input_kernel
    gaussian_symmetrized_kl_kernel
    grid_interpolation_kernel
    grid_kernel
    index_kernel
    inducing_point_kernel
    keops (package)
    kernel
    lcm_kernel
    linear_kernel
    matern_kernel
    multi_device_kernel
    multitask_kernel
    newton_girard_additive_kernel
    periodic_kernel
    piecewise_polynomial_kernel
    polynomial_kernel
    polynomial_kernel_grad
    product_structure_kernel
    rbf_kernel
    rbf_kernel_grad
    rff_kernel
    rq_kernel
    scale_kernel
    spectral_delta_kernel
    spectral_mixture_kernel

CLASSES
    gpytorch.module.Module(torch.nn.modules.module.Module)
        gpytorch.kernels.kernel.Kernel
            gpytorch.kernels.additive_structure_kernel.AdditiveStructureKernel
            gpytorch.kernels.arc_kernel.ArcKernel
     